In [2]:
from collections import defaultdict, deque
from openai import OpenAI
from typing import Dict, Any, Optional
import datetime
import string
import random
import os
from dataclasses import dataclass

# --- Translation helper ---
#Chong Ming 3
#This is a generic translation function to use the LLM as a translator.
def translate(client, model, text, source_lang, target_lang):
    if source_lang == target_lang:
        return text
    prompt = f"Translate this text from {source_lang} to {target_lang}:\n{text}"
    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return completion.choices[0].message.content.strip()
    
@dataclass
class NPCConfig:
    world_description: str
    character_description: str
    history_size: int = 10
    has_scratchpad: bool = False

class NPCFactoryError(Exception):
    """Base exception class for NPC Factory errors."""
    pass

class NPCNotFoundError(NPCFactoryError):
    """Raised when trying to interact with a non-existent NPC."""
    def __init__(self, npc_id: str):
        self.npc_id = npc_id
        super().__init__(f"NPC with ID '{npc_id}' not found")



In [3]:
class SimpleChatNPC:
    def __init__(self, client: OpenAI, model: str, config: NPCConfig):
        self.client = client
        self.model = model
        self.config = config
        self.chat_histories = defaultdict(lambda: deque(maxlen=config.history_size))

    def get_system_message(self) -> Dict[str, str]:
        """Returns the system message that defines the NPC's behavior."""
        character_description = self.config.character_description

        if self.config.has_scratchpad:
            character_description += """
You can use scratchpad for thinking before you answer: whatever you output between #SCRATCHPAD and #ANSWER won't be shown to anyone.
You start your output with #SCRATCHPAD and after you've done thinking, you #ANSWER"""

        return {
            "role": "system",
            "content": f"""WORLD SETTING: {self.config.world_description}
###
{character_description}"""
        }

    def chat(self, user_message: str, user_id: str) -> str:
        """Process a user message and return the NPC's response."""
        messages = [self.get_system_message()]

        # Add conversation history
        history = list(self.chat_histories[user_id])
        if history:
            messages.extend(history)

        # Add new user message
        user_message_dict = {
            "role": "user",
            "content": user_message
        }
        self.chat_histories[user_id].append(user_message_dict)
        messages.append(user_message_dict)

        try:
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.6
            )

            response = completion.choices[0].message.content

            # Handle scratchpad if enabled
            response_clean = response
            if self.config.has_scratchpad:
                import re
                scratchpad_match = re.search(r"#SCRATCHPAD(:?)(.*?)#ANSWER(:?)", response, re.DOTALL)
                if scratchpad_match:
                    response_clean = response[scratchpad_match.end():].strip()


            # Store response in history, including the scratchpad
            self.chat_histories[user_id].append({
                "role": "assistant",
                "content": response
            })

            # Return the message to the user without a scratchpad
            return response_clean

        except Exception as e:
            return f"Error: {str(e)}"

In [4]:
class NPCFactory:
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model
        self.npcs: Dict[str, SimpleChatNPC] = {}
        self.user_ids: Dict[str, str] = {}  # username -> user_id mapping
        '''
        Chong Ming 1:
        add language tracking in step 1 ("registering" or "setting" the user’s preferred language) 
        so the system knows what language each user wants to communicate in because different users might want to chat in different languages 
        In the code, use a dictionary:
        self.user_languages: Dict[str, str] = {}
        This maps each user_id to their chosen language, e.g.,
        { 'user123': 'French', 'user456': 'German' }
        
        When the user is registered or their language is set, I save it:
        npc_factory.set_user_language(user_id, "French")
        
        When processing a chat:
        Incoming: Translate from their language to English.
        Outgoing: Translate from English back to their language.
        '''
        self.user_languages: Dict[str, str] = {}  # <-- CHONGMING 1: Add this line

    def generate_id(self) -> str:
        """Generate a random unique identifier."""
        return ''.join(random.choice(string.ascii_letters) for _ in range(8))

    def register_user(self, username: str) -> str:
        """Register a new user and return their unique ID.
        If username already exists, appends a numerical suffix."""
        base_username = username
        suffix = 1

        # Keep trying with incremented suffixes until we find an unused name
        while username in self.user_ids:
            username = f"{base_username}_{suffix}"
            suffix += 1

        user_id = self.generate_id()
        self.user_ids[username] = user_id
        return user_id

    def set_user_language(self, user_id: str, language: str):
        """Set the preferred language for a user."""
        ### YOUR CODE HERE
        self.user_languages[user_id] = language # <-- CHONGMING 2: Add this line

    def register_npc(self, world_description: str, character_description: str,
                     history_size: int = 10, has_scratchpad: bool = False) -> str:
        """Create and register a new NPC, returning its unique ID."""
        npc_id = self.generate_id()

        config = NPCConfig(
            world_description=world_description,
            character_description=character_description,
            history_size=history_size,
            has_scratchpad=has_scratchpad
        )

        self.npcs[npc_id] = SimpleChatNPC(self.client, self.model, config)
        return npc_id

    def chat_with_npc(self, npc_id: str, user_id: str, message: str) -> str:
        """Send a message to a specific NPC from a specific user.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user
            message: The message to send

        Returns:
            The NPC's response

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        npc = self.npcs[npc_id]
        # CHONGMING 5
        #Add in these lines for localization
        '''
        Takes the user's original message (which could be in any language)
        Uses the localize_input function to translate it into English (the language the LLM expects)
        Result: message_en is the user's message in English
        '''
        message_en = self.localize_input(user_id, message) #<-- CHONGMING 5: Add this line

        '''
        Passes the translated English message (message_en) to the NPC's chat function
        The NPC/LLM processes the conversation in English and generates a response in English
        Result: response_en is the NPC's answer, still in English
        '''
        response_en = npc.chat(message_en, user_id) #<-- CHONGMING 5: Add this line
        
        '''
        Takes the NPC’s English response (response_en)
        Uses the localize_output function to translate it back into the user's preferred language (as tracked by their user_id)
        Result: response_localized is the NPC's answer in the user's own language
        '''
        response_localized = self.localize_output(user_id, response_en) #<-- CHONGMING 5: Add this line
        
        #Returns the translated response to the user, so they get a reply in their original language
        return response_localized #<-- CHONGMING 5: Add this line
        #return npc.chat(message, user_id)

    def get_npc_chat_history(self, npc_id: str, user_id: str) -> list:
        """Retrieve chat history between a specific user and NPC.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user

        Returns:
            List of message dictionaries containing the chat history

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        return list(self.npcs[npc_id].chat_histories[user_id])
    
    #Chong Ming 4
    #Add a localize_input and localize_output method to handle translation.
    '''
    CHONG MING 4
    The main purpose of this is to enable the NPC logic (which was designed to operate in English) to work correctly, 
    no matter what language the user prefers.
    User sends a message in their preferred language (eg: French).
    System detects the user's language from tracking (step 1).
    System translates the user's message into English (eg:"Hello, who are you?").
    The NPC processes the English message and generates an English response as it normally would.
    '''
    def localize_input(self, user_id, text):
        user_lang = self.user_languages.get(user_id, "English")
        return translate(self.client, self.model, text, user_lang, "English")

    def localize_output(self, user_id, text):
        user_lang = self.user_languages.get(user_id, "English")
        return translate(self.client, self.model, text, "English", user_lang)

## Connect to Nebius LLM service

In [5]:
with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

from openai import OpenAI

# Nebius uses the same OpenAI() class, but with additional details
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

model = "meta-llama/Meta-Llama-3.1-405B-Instruct"

# Creating a factory
npc_factory = NPCFactory(client=client, model=model)

## To format the LLM output to be more readable

In [7]:
def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n")
    for line in lines:
        current_line = ""
        words = line.split()
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

In [13]:
# Register a user
user_id = npc_factory.register_user("Alice")

# Set user preffered language
preffered_language = "French"
npc_factory.set_user_language(user_id, preffered_language)

# Create an NPC
npc_id = npc_factory.register_npc(
    world_description="""Modern financial centers from Singapore to London are powered by sophisticated transaction banking networks. 
                         Payments, collections, and liquidity management are no longer manual; 
                         AI-driven systems and cloud platforms automate most treasury tasks. 
                         Corporates expect instant, transparent cross-border payments and intelligent cash pooling. 
                         The role of transaction banking is more critical than ever to support global supply chains, manage risk, 
                         and fuel growth in a volatile economic landscape """,
    
    character_description="You are a Payments Product Manager working in Transaction Banking",
    has_scratchpad=True
)

In [14]:
response = npc_factory.chat_with_npc(npc_id, user_id,
                                     """qu'est-ce que le paiement en gros"""
                                     )
print(prettify_string(response))

Paiement wholesale, également connu sous le nom de paiement en vrac, fait
référence à un type de paiement qui implique un grand nombre de transactions,
souvent regroupées et traitées en une seule fois, généralement utilisé pour les
transactions de business à business (B2B), comme les paiements de salaires, les
paiements des fournisseurs ou les encaissements.

However, please note that 'wholesale payment' is more approximately translated
to 'paiement de gros' in French. So the more proper translation would be:

Paiement de gros, également connu sous le nom de paiement en vrac, fait
référence à un type de paiement qui implique un grand nombre de transactions,
souvent regroupées et traitées en une seule fois, généralement utilisé pour les
transactions de business à business (B2B), comme les paiements de salaires, les
paiements des fournisseurs ou les encaissements.


In [15]:
npc_factory.get_npc_chat_history(npc_id, user_id)

[{'role': 'user',
  'content': 'The translation of the text from French to English is:\n\n"What is wholesale payment" or "What is bulk payment".'},
 {'role': 'assistant',
  'content': "#SCRATCHPAD\nTo answer this question, I need to put on my Payments Product Manager hat. \n\nIn transaction banking, we often deal with large volumes of transactions from corporate clients. Wholesale or bulk payments refer to a type of payment that involves a large number of transactions, often batched together and processed in one go. \n\nThese payments are typically used for business-to-business (B2B) transactions, such as salary payments, supplier payments, or collections. Wholesale payments are often high-value and high-volume, requiring specialized processing and settlement arrangements.\n\nAs a Payments Product Manager, I would work with our corporate clients to understand their wholesale payment needs, design payment solutions that meet their requirements, and ensure seamless execution through our 